In [1]:
import pandas as pd
import numpy as np
import os

In [6]:
new_df = pd.read_csv("sampleData.csv")
new_df.head()

,Unnamed: 0.1,Unnamed: 0_x,Tweets_x,len_x
0,0,1,thereidout blog karine jean pierre can do wha...,19
1,1,2,nyc mayor using florida billboards in attempt ...,13
2,2,3,sen hagerty china cartels partnering to create...,19
3,3,4,these are america favorite pickup trucks,10
4,4,5,watch live president biden vice president harr...,23


In [37]:
blp = pd.read_excel('BLP summary data words.xls')
blp.set_index('spelling', inplace=True)
print(blp.shape)
blp.head()

(28730, 18)


,rt,zscore,accuracy,zscore.sd,SUBTLEX,bnc.frequency,coltheart.N,OLD20,stress,nletters,nsyl,celex.frequency,celex.frequency.lemma,ELPldtrt,ELPldtzrt,ELPldtacc,ELPnamrt,ELPnamzrt
spelling,,,,,,,,,,,,,,,,,,
aas,NaN,NaN,0.050,NaN,2.0,9.0,6,1.55,10.0,3.0,2.0,0.0,52.0,NaN,NaN,NaN,NaN,NaN
aback,647.00,0.370296,0.400,1.179214,15.0,327.0,2,1.85,0.0,5.0,2.0,59.0,59.0,796.27,0.11,0.45,596.54,-0.55
abaft,1376.00,2.404425,0.025,NaN,2.0,8.0,0,2.00,0.0,5.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
abase,692.75,0.437372,0.100,0.514838,3.0,6.0,3,1.70,0.0,5.0,2.0,0.0,2.0,803.27,0.03,0.69,640.21,-0.37
abased,695.75,0.912018,0.250,0.713795,NaN,6.0,3,1.75,0.0,6.0,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN


In [38]:
average_response_time = blp.rt.mean()

In [39]:
average_response_time

654.4973715548408

In [49]:
chunk_size = 10000

In [53]:
saved_files = [filename for filename in os.listdir('.') if filename.startswith('intermediate_results_')]
if saved_files:

    saved_files.sort()

    last_saved_file = saved_files[-1]
    result_df = pd.read_csv(last_saved_file)

    last_processed_index = result_df.index[-1] + 1
else:

    result_df = pd.DataFrame()

    last_processed_index = 0

for chunk_start in range(last_processed_index, len(new_df), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(new_df))
    chunk = new_df.iloc[chunk_start:chunk_end]

    chunk['sum_response_time'] = chunk['Tweets'].apply(lambda x: np.sum([blp.loc[w, 'rt'] for w in x.split() if w in blp.index]) if len([blp.loc[w, 'rt'] for w in x.split() if w in blp.index]) > 0 else average_response_time)

    chunk['mean_response_time'] = chunk['sum_response_time'] / chunk['len']

    result_df = pd.concat([result_df, chunk[['sum_response_time', 'mean_response_time']]])

    if (chunk_start + chunk_size) % (10 * chunk_size) == 0:
        result_df.to_csv(f'intermediate_results_{chunk_start + chunk_size}.csv', index=False)

result_df.head

/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_90406/3447868251.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['sum_response_time'] = chunk['Tweets'].apply(lambda x: np.sum([blp.loc[w, 'rt'] for w in x.split() if w in blp.index]) if len([blp.loc[w, 'rt'] for w in x.split() if w in blp.index]) > 0 else average_response_time)
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_90406/3447868251.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['mean_response_time'] = chu

<bound method NDFrame.head of          sum_response_time  mean_response_time
0              4583.625017          241.243422
1              3868.552096          297.580930
2              5684.513669          299.184930
3              1733.549571          173.354957
4              5417.119383          235.526930
...                    ...                 ...
1383515        3989.190337          173.443058
1383516        5748.421274          261.291876
1383517        2104.078674          123.769334
1383518        3357.209007          139.883709
1383519        2597.607287          236.146117

[1383520 rows x 2 columns]>

In [54]:
combined_df = pd.concat([new_df, result_df[['sum_response_time', 'mean_response_time']]], axis=1)
combined_df

,Unnamed: 0.1,Unnamed: 0,Conversation_id,Author_id,Tweet,Tweets,Retweet,len,sum_arousal,mean_arousal,len_orthogonalized,sum_arousal_orthogonalized,mean_arousal_orthogonalized,sum_response_time,mean_response_time
0,0,1,1.524918e+18,2836421.0,.@thereidout Blog: Karine Jean-Pierre can do w...,thereidout blog karine jean pierre can do wha...,16,19,22.77,1.751538,0.000738,6.956677e-04,0.000179,4583.625017,241.243422
1,1,2,1.511412e+18,1367531.0,NYC mayor using Florida billboards in attempt ...,nyc mayor using florida billboards in attempt ...,24,13,17.27,1.727000,0.000692,8.791377e-04,0.000610,3868.552096,297.580930
2,2,3,1.511406e+18,1367531.0,"Sen. Hagerty: China, cartels partnering to cre...",sen hagerty china cartels partnering to create...,28,19,25.58,1.967692,0.000687,9.037052e-04,0.000190,5684.513669,299.184930
3,3,4,1.511407e+18,1367531.0,These are America's favorite pickup trucks htt...,these are america favorite pickup trucks,16,10,8.25,1.375000,0.000886,8.879085e-08,0.001445,1733.549571,173.354957
4,4,5,1.511408e+18,1367531.0,"WATCH LIVE: President Biden, Vice President Ha...",watch live president biden vice president harr...,13,23,41.89,2.327222,0.000555,1.394977e-03,-0.000217,5417.119383,235.526930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383515,1383515,1383516,1.666551e+17,2836421.0,Eastwood's #SB46 ad for Chrysler turning into ...,eastwood sb46 ad for chrysler turning into pol...,19,23,18.93,1.262000,0.000891,-4.306603e-06,-0.000009,3989.190337,173.443058
1383516,1383516,1383517,1.665653e+17,2836421.0,Chrysler's 'Eastwood' is leading our viewer vo...,chrysler eastwood is leading our viewer vote o...,17,22,8.62,0.507059,0.001075,-1.174325e-03,-0.000086,5748.421274,261.291876
1383517,1383517,1383518,1.665436e+17,2836421.0,"#SuperBowl breaks Twitter record (Sorry, #Tebo...",superbowl breaks twitter record sorry tebow h...,29,17,7.92,0.792000,0.001046,-9.471502e-04,0.000305,2104.078674,123.769334
1383518,1383518,1383519,1.663623e+17,2836421.0,"""Here Weego?"" ""Broderick, Broderick?"" Which #S...",here weego broderick broderick which superbow...,10,24,3.33,0.237857,0.001143,-1.979549e-03,-0.000013,3357.209007,139.883709


In [55]:
combined_df.to_csv("new_data0712.csv", index=False)

In [17]:
combined_df = pd.read_csv("new_data0712.csv")
combined_df

,Unnamed: 0.1,Unnamed: 0,Conversation_id,Author_id,Tweet,Tweets,Retweet,len,sum_arousal,mean_arousal,len_orthogonalized,sum_arousal_orthogonalized,mean_arousal_orthogonalized,sum_response_time,mean_response_time
0,0,1,1.524918e+18,2836421.0,.@thereidout Blog: Karine Jean-Pierre can do w...,thereidout blog karine jean pierre can do wha...,16,19,22.77,1.751538,0.000738,6.956677e-04,0.000179,4583.625017,241.243422
1,1,2,1.511412e+18,1367531.0,NYC mayor using Florida billboards in attempt ...,nyc mayor using florida billboards in attempt ...,24,13,17.27,1.727000,0.000692,8.791377e-04,0.000610,3868.552096,297.580930
2,2,3,1.511406e+18,1367531.0,"Sen. Hagerty: China, cartels partnering to cre...",sen hagerty china cartels partnering to create...,28,19,25.58,1.967692,0.000687,9.037052e-04,0.000190,5684.513669,299.184930
3,3,4,1.511407e+18,1367531.0,These are America's favorite pickup trucks htt...,these are america favorite pickup trucks,16,10,8.25,1.375000,0.000886,8.879085e-08,0.001445,1733.549571,173.354957
4,4,5,1.511408e+18,1367531.0,"WATCH LIVE: President Biden, Vice President Ha...",watch live president biden vice president harr...,13,23,41.89,2.327222,0.000555,1.394977e-03,-0.000217,5417.119383,235.526930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383515,1383515,1383516,1.666551e+17,2836421.0,Eastwood's #SB46 ad for Chrysler turning into ...,eastwood sb46 ad for chrysler turning into pol...,19,23,18.93,1.262000,0.000891,-4.306603e-06,-0.000009,3989.190337,173.443058
1383516,1383516,1383517,1.665653e+17,2836421.0,Chrysler's 'Eastwood' is leading our viewer vo...,chrysler eastwood is leading our viewer vote o...,17,22,8.62,0.507059,0.001075,-1.174325e-03,-0.000086,5748.421274,261.291876
1383517,1383517,1383518,1.665436e+17,2836421.0,"#SuperBowl breaks Twitter record (Sorry, #Tebo...",superbowl breaks twitter record sorry tebow h...,29,17,7.92,0.792000,0.001046,-9.471502e-04,0.000305,2104.078674,123.769334
1383518,1383518,1383519,1.663623e+17,2836421.0,"""Here Weego?"" ""Broderick, Broderick?"" Which #S...",here weego broderick broderick which superbow...,10,24,3.33,0.237857,0.001143,-1.979549e-03,-0.000013,3357.209007,139.883709


In [ ]:
df11 = pd.read_csv("new_data0712.csv")
df11

In [8]:
!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [13]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/xiaoqin/nltk_data...


True

In [15]:
sia = SentimentIntensityAnalyzer()
sentiment_df = new_df[['Tweets_x']].apply(lambda x: sia.polarity_scores(x[0]), axis=1, result_type='expand')
sentiment_df.columns = ['vader_'+x for x in sentiment_df.columns]
new_df = pd.concat([new_df, sentiment_df], axis=1)
new_df['vader_isnegative'] = new_df['vader_compound'].apply(lambda x: int(x<=-0.05))
new_df['vader_ispositive'] = new_df['vader_compound'].apply(lambda x: int(x>=0.05))
new_df['vader_isneutral'] = new_df['vader_compound'].apply(lambda x: int((x<0.05) and (x>-0.05)))

In [18]:
new_df.to_csv("vadercolumns.csv", index=False)

In [20]:
new_df

,Unnamed: 0.1,Unnamed: 0_x,Tweets_x,len_x,vader_neg,vader_neu,vader_pos,vader_compound,vader_isnegative,vader_ispositive,vader_isneutral
0,0,1,thereidout blog karine jean pierre can do wha...,19,0.000,1.000,0.000,0.0000,0,0,1
1,1,2,nyc mayor using florida billboards in attempt ...,13,0.000,1.000,0.000,0.0000,0,0,1
2,2,3,sen hagerty china cartels partnering to create...,19,0.000,0.851,0.149,0.2732,0,1,0
3,3,4,these are america favorite pickup trucks,10,0.000,0.625,0.375,0.4588,0,1,0
4,4,5,watch live president biden vice president harr...,23,0.000,0.842,0.158,0.4939,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1382573,1383515,1383516,eastwood sb46 ad for chrysler turning into pol...,23,0.000,0.881,0.119,0.2263,0,1,0
1382574,1383516,1383517,chrysler eastwood is leading our viewer vote o...,22,0.000,0.847,0.153,0.4404,0,1,0
1382575,1383517,1383518,superbowl breaks twitter record sorry tebow h...,17,0.126,0.874,0.000,-0.0772,1,0,0
1382576,1383518,1383519,here weego broderick broderick which superbow...,24,0.000,0.756,0.244,0.6369,0,1,0


In [3]:
#orthogonalized sum_response_time	mean_response_time
df5 = pd.read_csv("sampledata.csv")
df5

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x_x,len_x_x,sum_response_time,mean_response_time
0,0,0,1,19,4583.625017,241.243422
1,1,1,2,13,3868.552096,297.580930
2,2,2,3,19,5684.513669,299.184930
3,3,3,4,10,1733.549571,173.354957
4,4,4,5,23,5417.119383,235.526930
...,...,...,...,...,...,...
1382573,1382573,1383515,1383516,23,3989.190337,173.443058
1382574,1382574,1383516,1383517,22,5748.421274,261.291876
1382575,1382575,1383517,1383518,17,2104.078674,123.769334
1382576,1382576,1383518,1383519,24,3357.209007,139.883709


In [5]:
def modified_gs(A: np.ndarray, weights: np.ndarray) -> np.ndarray:
    num_vecs = A.shape[0]
    num_dims = A.shape[1]

    L = np.zeros((num_vecs, 1))  
    for i in range(num_vecs):
        L[i] = np.sqrt(weights[i] * (A[i].T @ A[i]))

    V = A.copy() / L
    B = V.copy()
    for j in range(0, num_vecs):
        B[j] = V[j] / np.sqrt(V[j].T @ V[j])
        for k in range(j, num_vecs):
            V[k] = V[k] - (B[j].T @ V[k]) * B[j]
    return B

In [10]:
filename = "sampledata.csv"

In [11]:
chunksize = 10000

In [12]:
try:
    last_processed_row = pd.read_csv("last_processed_row.txt", header=None)[0][0]
except FileNotFoundError:
    last_processed_row = 0


for chunk in pd.read_csv(filename, chunksize=chunksize, skiprows=range(1, last_processed_row + 1)):
    
    data = chunk[['sum_response_time', 'mean_response_time']].to_numpy()
    weights = chunk['len_x_x'].to_numpy().reshape(-1, 1)
    orthogonalized_data = modified_gs(data, weights)
    
    
    chunk['sum_response_time_orthogonalized'] = orthogonalized_data[:, 0]
    chunk['mean_response_time_orthogonalized'] = orthogonalized_data[:, 1]
    chunk['len_x_x_orthogonalized'] = chunk['len_x_x']
    
    
    with open('orthogonalized.csv', 'a') as f:
        chunk.to_csv(f, index=False, header=f.tell() == 0)
    
   
    with open("last_processed_row.txt", "w") as f:
        f.write(str(chunk.index[-1]))

/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_96520/726420228.py:12: RuntimeWarning: invalid value encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_96520/726420228.py:12: RuntimeWarning: invalid value encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_96520/726420228.py:12: RuntimeWarning: invalid value encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_96520/726420228.py:12: RuntimeWarning: invalid value encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_96520/726420228.py:12: RuntimeWarning: invalid value encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_96520/726420228.py:12: RuntimeWarning: invalid valu

In [13]:
df6 = pd.read_csv("orthogonalized.csv")
df6

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x_x,len_x_x,sum_response_time,mean_response_time,sum_response_time_orthogonalized,mean_response_time_orthogonalized,len_x_x_orthogonalized
0,0,0,1,19,4583.625017,241.243422,0.998618,0.052559,19
1,1,1,2,13,3868.552096,297.580930,-0.052559,0.998618,13
2,2,2,3,19,5684.513669,299.184930,0.998618,0.052559,19
3,3,3,4,10,1733.549571,173.354957,0.052559,-0.998618,10
4,4,4,5,23,5417.119383,235.526930,0.998712,0.050728,23
...,...,...,...,...,...,...,...,...,...
1382573,1382573,1383515,1383516,23,3989.190337,173.443058,NaN,NaN,23
1382574,1382574,1383516,1383517,22,5748.421274,261.291876,NaN,NaN,22
1382575,1382575,1383517,1383518,17,2104.078674,123.769334,NaN,NaN,17
1382576,1382576,1383518,1383519,24,3357.209007,139.883709,NaN,NaN,24


In [14]:
#orthogonalized sum_arousal_x	mean_arousal_x
df7 = pd.read_csv("sampledata.csv")
df7

,Unnamed: 0.1,Unnamed: 0_x_x_x,len_x_x_x,sum_arousal_x,mean_arousal_x
0,0,1,19,22.77,1.751538
1,1,2,13,17.27,1.727000
2,2,3,19,25.58,1.967692
3,3,4,10,8.25,1.375000
4,4,5,23,41.89,2.327222
...,...,...,...,...,...
1382573,1383515,1383516,23,18.93,1.262000
1382574,1383516,1383517,22,8.62,0.507059
1382575,1383517,1383518,17,7.92,0.792000
1382576,1383518,1383519,24,3.33,0.237857


In [6]:
filename = "sampledata.csv"

In [7]:
chunksize = 10000

In [8]:
try:
    last_processed_row = pd.read_csv("last_processed_row.txt", header=None)[0][0]
except FileNotFoundError:
    last_processed_row = 0


for chunk in pd.read_csv(filename, chunksize=chunksize, skiprows=range(1, last_processed_row + 1)):
    
    data = chunk[['sum_arousal_x', 'mean_arousal_x']].to_numpy()
    weights = chunk['len_x_x_x'].to_numpy().reshape(-1, 1)
    orthogonalized_data = modified_gs(data, weights)
    
    
    chunk['sum_arousal_orthogonalized'] = orthogonalized_data[:, 0]
    chunk['mean_arousal_orthogonalized'] = orthogonalized_data[:, 1]
    chunk['len_x_x_x_orthogonalized'] = chunk['len_x_x_x']
    
    
    with open('orthogonalized2.csv', 'a') as f:
        chunk.to_csv(f, index=False, header=f.tell() == 0)
    
   
    with open("last_processed_row.txt", "w") as f:
        f.write(str(chunk.index[-1]))

/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_50742/250117572.py:9: RuntimeWarning: invalid value encountered in true_divide
  V = A.copy() / L
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_50742/250117572.py:12: RuntimeWarning: divide by zero encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_50742/250117572.py:14: RuntimeWarning: invalid value encountered in matmul
  V[k] = V[k] - (B[j].T @ V[k]) * B[j]
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_50742/250117572.py:9: RuntimeWarning: invalid value encountered in true_divide
  V = A.copy() / L
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_50742/250117572.py:12: RuntimeWarning: invalid value encountered in true_divide
  B[j] = V[j] / np.sqrt(V[j].T @ V[j])
/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_50742/250117572.py:9: RuntimeWarning: invalid value encountered in true_divide
  V = A.copy() / L

In [9]:
df8 = pd.read_csv("orthogonalized2.csv")
df8

,Unnamed: 0.1,Unnamed: 0_x_x_x,len_x_x_x,sum_arousal_x,mean_arousal_x,sum_arousal_orthogonalized,mean_arousal_orthogonalized,len_x_x_x_orthogonalized
0,1383519,1383520,11,12.27,1.227000,0.995037,0.099504,11
1,0,1,19,22.77,1.751538,0.997054,0.076696,19
2,1,2,13,17.27,1.727000,-0.076696,0.997054,13
3,2,3,19,25.58,1.967692,0.997054,0.076696,19
4,3,4,10,8.25,1.375000,0.076696,-0.997054,10
...,...,...,...,...,...,...,...,...
2482575,1383515,1383516,23,18.93,1.262000,NaN,NaN,23
2482576,1383516,1383517,22,8.62,0.507059,NaN,NaN,22
2482577,1383517,1383518,17,7.92,0.792000,NaN,NaN,17
2482578,1383518,1383519,24,3.33,0.237857,NaN,NaN,24
